In [15]:
%matplotlib inline

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('./python')
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/coco_caption
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_mode_gpu()

sys.path.append('./examples/char-rnn')

In [17]:
!head ../../examples/char-rnn/vocabulary.txt

F
i
r
s
t
 
C
z
e
n


In [26]:
vocabulary = ['<EOS>'] + [line.strip() for line in
                          open('../../examples/char-rnn/vocab_shakespeare.txt').readlines()]
print len(vocabulary)

66


In [27]:
iter_num = 17900
net = caffe.Net('../../examples/char-rnn/char-rnn_deploy.prototxt',
                '../../examples/char-rnn/char-rnn_iter_%d.caffemodel' % iter_num, caffe.TEST)
print net.blobs['probs'].data.shape

(1, 1, 66)


In [28]:
def softmax(softmax_inputs, temp):
    shifted_inputs = softmax_inputs - softmax_inputs.max()
    exp_outputs = np.exp(temp * shifted_inputs)
    exp_outputs_sum = exp_outputs.sum()
    if np.isnan(exp_outputs_sum):
        return exp_outputs * float('nan')
    assert exp_outputs_sum > 0
    if np.isinf(exp_outputs_sum):
        return np.zeros_like(exp_outputs)
    eps_sum = 1e-20
    return exp_outputs / max(exp_outputs_sum, eps_sum)

def random_choice_from_probs(softmax_inputs, temp=1):
    # temperature of infinity == take the max
    if temp == float('inf'):
        return np.argmax(softmax_inputs)
    probs = softmax(softmax_inputs, temp)
    r = random.random()
    cum_sum = 0.
    for i, p in enumerate(probs):
        cum_sum += p
        if cum_sum >= r: return i
    return 1  # return UNK?

In [29]:
def generate_sentence(net, input_char, temp=float('inf'), output='predict', max_words=200):
    cont_input = np.array([0])
    word_input = np.array([vocabulary.index(input_char)])
    sentence = []
    while len(sentence) < max_words and (not sentence or sentence[-1] != 0):
        net.forward(cont=cont_input, input=word_input)
        output_preds = net.blobs[output].data[0, 0, :]
        sentence.append(random_choice_from_probs(output_preds, temp=temp))
        cont_input[0] = 1
        word_input[0] = sentence[-1]
    return sentence

def charstostr(char_arr):
    out = ''
    for i in [vocabulary[index] for index in char_arr]:
        if i == '':
            i = ' '
        if i == '\\n':
            i = '\n'
        out += i
        
    return out

In [30]:
choice = random.randint(0, len(vocabulary) - 1)
start_char = vocabulary[choice]
print 'Chosen %s as startup char with idx %d' % (start_char, choice)

Chosen q as startup char with idx 55


In [31]:
sentence = generate_sentence(net, start_char)
print sentence
print charstostr(sentence)

[22, 2, 5, 6, 5, 23, 9, 6, 16, 15, 3, 29, 19, 24, 12, 28, 10, 19, 6, 5, 23, 9, 3, 9, 14, 15, 3, 9, 6, 39, 6, 4, 23, 20, 29, 29, 6, 43, 9, 6, 4, 15, 6, 43, 9, 20, 3, 6, 5, 23, 9, 6, 4, 9, 20, 12, 28, 10, 19, 6, 5, 23, 9, 6, 4, 5, 20, 5, 9, 6, 15, 14, 6, 5, 23, 9, 6, 18, 15, 22, 3, 5, 24, 6, 20, 10, 19, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 4, 23, 9, 12, 16, 2, 29, 29, 6, 4, 9, 9, 6, 5, 23, 9, 6, 4, 5, 20, 5, 9, 6, 15, 14, 6, 5, 23, 9, 6, 18, 15, 22, 3, 5, 24, 6, 20, 10, 19, 6, 5, 23, 9, 6, 4, 9, 20, 12, 45, 14, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 12]
uit the world,
And therefore I shall be so bear the sea
And the state of the court, and the sea that she
will see the state of the court, and the sea
Of the sea that the sea that the sea that the sea



In [32]:
sentence = generate_sentence(net, start_char)
print sentence
print charstostr(sentence)

[22, 2, 5, 6, 5, 23, 9, 6, 16, 15, 3, 29, 19, 24, 12, 28, 10, 19, 6, 5, 23, 9, 3, 9, 14, 15, 3, 9, 6, 39, 6, 4, 23, 20, 29, 29, 6, 43, 9, 6, 4, 15, 6, 43, 9, 20, 3, 6, 5, 23, 9, 6, 4, 9, 20, 12, 28, 10, 19, 6, 5, 23, 9, 6, 4, 5, 20, 5, 9, 6, 15, 14, 6, 5, 23, 9, 6, 18, 15, 22, 3, 5, 24, 6, 20, 10, 19, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 4, 23, 9, 12, 16, 2, 29, 29, 6, 4, 9, 9, 6, 5, 23, 9, 6, 4, 5, 20, 5, 9, 6, 15, 14, 6, 5, 23, 9, 6, 18, 15, 22, 3, 5, 24, 6, 20, 10, 19, 6, 5, 23, 9, 6, 4, 9, 20, 12, 45, 14, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 6, 5, 23, 20, 5, 6, 5, 23, 9, 6, 4, 9, 20, 12]
uit the world,
And therefore I shall be so bear the sea
And the state of the court, and the sea that she
will see the state of the court, and the sea
Of the sea that the sea that the sea that the sea



In [33]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

[22, 2, 5, 24, 12, 49, 23, 20, 5, 6, 16, 23, 2, 18, 23, 6, 39, 6, 19, 15, 6, 43, 22, 5, 6, 23, 20, 17, 17, 21, 6, 20, 6, 4, 2, 29, 32, 9, 3, 6, 16, 23, 2, 18, 23, 6, 2, 14, 6, 5, 23, 15, 22, 6, 4, 9, 2, 8, 9, 12, 43, 9, 6, 20, 29, 3, 9, 20, 19, 21, 11, 6, 20, 10, 19, 6, 43, 9, 2, 10, 41, 6, 5, 15, 6, 5, 23, 9, 6, 16, 20, 3, 6, 15, 14, 6, 5, 23, 2, 4, 27, 12, 12, 53, 52, 59, 51, 6, 47, 39, 40, 7, 51, 40, 49, 39, 45, 11, 12, 58, 15, 24, 6, 18, 23, 2, 29, 19, 24, 6, 20, 4, 6, 5, 23, 15, 22, 6, 4, 23, 20, 29, 5, 6, 5, 23, 3, 2, 32, 9, 6, 5, 15, 6, 7, 2, 18, 9, 14, 22, 4, 9, 6, 16, 23, 2, 18, 23, 6, 39, 6, 25, 20, 21, 12, 13, 9, 2, 10, 41, 6, 5, 15, 6, 5, 23, 9, 6, 17, 3, 9, 4, 9, 10, 5, 6, 41, 2, 32, 9]
uit,
That which I do but happy a silver which if thou seize
be already: and being to the war of this.

DUKE VINCENTIO:
Go, child, as thou shalt thrive to Cicefuse which I may
Being to the present give


In [34]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

[22, 2, 5, 24, 6, 2, 10, 6, 23, 20, 10, 19, 4, 12, 39, 10, 6, 20, 29, 29, 6, 5, 23, 9, 6, 25, 2, 19, 24, 6, 2, 4, 29, 9, 6, 20, 16, 20, 21, 24, 6, 14, 15, 3, 6, 4, 23, 3, 22, 4, 4, 24, 12, 7, 15, 25, 14, 15, 3, 5, 6, 23, 20, 5, 23, 6, 5, 23, 9, 6, 19, 9, 32, 2, 18, 9, 6, 5, 15, 6, 5, 23, 21, 6, 4, 29, 20, 22, 41, 23, 5, 9, 3, 4, 11, 6, 23, 15, 16, 6, 10, 2, 41, 23, 5, 12, 36, 23, 9, 10, 6, 18, 3, 2, 9, 4, 6, 54, 20, 22, 29, 2, 10, 20, 27, 12, 12, 7, 28, 35, 39, 38, 38, 45, 11, 12, 39, 6, 5, 23, 2, 10, 26, 24, 6, 20, 4, 6, 25, 15, 18, 26, 37, 19, 6, 16, 2, 5, 23, 6, 18, 3, 9, 17, 5, 6, 20, 10, 19, 6, 41, 3, 2, 9, 32, 9, 6, 16, 15, 3, 5, 23, 21, 6, 5, 9, 20, 3, 4, 42, 12, 13, 9, 6, 5, 23, 15, 22, 6, 23]
uit, in hands
In all the mid, isle away, for shruss,
Comfort hath the device to thy slaughters: how night
When cries Paulina.

CAMILLO:
I think, as mock'd with crept and grieve worthy tears;
Be thou h


In [12]:
sentence = generate_sentence(net, start_char, temp=1.5)
print sentence
print charstostr(sentence)

[24, 12, 28, 4, 6, 2, 5, 6, 5, 23, 20, 5, 6, 14, 20, 3, 9, 16, 9, 29, 29, 27, 12, 12, 30, 39, 7, 39, 40, 39, 52, 30, 11, 12, 28, 4, 6, 25, 22, 18, 23, 6, 20, 4, 6, 21, 15, 22, 3, 6, 18, 23, 20, 25, 20, 18, 2, 10, 41, 6, 18, 15, 22, 10, 4, 9, 29, 24, 12, 28, 10, 19, 6, 23, 20, 32, 2, 10, 41, 6, 5, 23, 9, 6, 18, 15, 22, 10, 5, 21, 37, 4, 6, 4, 2, 10, 41, 22, 29, 20, 3, 29, 21, 24, 12, 49, 15, 6, 15, 56, 17, 3, 9, 4, 4, 37, 19, 6, 5, 23, 9, 6, 4, 2, 10, 41, 29, 9, 6, 14, 20, 2, 3, 24, 6, 4, 2, 3, 27, 12, 12, 7, 28, 35, 39, 38, 38, 45, 11, 12, 30, 15, 6, 16, 20, 4, 6, 9, 32, 9, 3, 6, 16, 20, 4, 6, 5, 23, 9, 3, 9, 6, 2, 10, 6, 5, 23, 9, 6, 17, 9, 15, 17, 29, 9, 24, 12, 36, 23, 15, 25, 6, 39, 6, 16, 2, 29, 29, 6, 10]
,
As it that farewell.

SICINIUS:
As much as your chamacing counsel,
And having the county's singularly,
To oxpress'd the single fair, sir.

CAMILLO:
So was ever was there in the people,
Whom I will n


In [13]:
sentence = generate_sentence(net, start_char, temp=1.5)
print sentence
print charstostr(sentence)

[27, 12, 12, 7, 45, 34, 39, 45, 38, 28, 40, 52, 30, 11, 12, 39, 6, 23, 20, 32, 9, 6, 5, 15, 6, 23, 20, 32, 9, 6, 5, 23, 9, 6, 25, 15, 3, 10, 2, 10, 41, 6, 5, 15, 6, 5, 23, 9, 6, 17, 20, 2, 10, 4, 12, 45, 14, 6, 5, 23, 20, 5, 6, 9, 32, 9, 3, 6, 23, 9, 20, 3, 6, 5, 23, 9, 6, 23, 9, 20, 3, 5, 6, 15, 14, 6, 4, 22, 43, 4, 5, 20, 10, 18, 9, 24, 12, 13, 22, 5, 6, 4, 23, 20, 21, 15, 16, 6, 5, 23, 9, 6, 4, 9, 10, 20, 5, 9, 6, 23, 15, 10, 15, 22, 3, 6, 29, 2, 26, 9, 6, 20, 6, 29, 2, 5, 5, 29, 9, 6, 17, 3, 15, 22, 19, 24, 12, 39, 6, 16, 2, 29, 29, 6, 3, 9, 4, 15, 29, 32, 9, 19, 6, 10, 15, 6, 23, 9, 20, 3, 5, 6, 14, 15, 3, 6, 5, 23, 9, 6, 14, 29, 15, 16, 9, 3, 4, 33, 6, 50, 9, 29, 17, 24, 6, 20, 10, 19, 12, 5]
.

CORIOLANUS:
I have to have the morning to the pains
Of that ever hear the heart of substance,
But shayow the senate honour like a little proud,
I will resolved no heart for the flowers? Help, and
t


In [14]:
sentence = generate_sentence(net, start_char, temp=3.0)
print sentence
print charstostr(sentence)

[6, 5, 23, 9, 10, 27, 12, 12, 1, 2, 3, 4, 5, 6, 7, 2, 5, 2, 8, 9, 10, 11, 12, 49, 23, 9, 6, 4, 23, 20, 25, 9, 6, 4, 23, 20, 29, 29, 6, 43, 9, 6, 25, 21, 6, 19, 9, 20, 5, 23, 6, 15, 14, 6, 20, 6, 23, 15, 3, 4, 9, 11, 12, 49, 23, 9, 6, 3, 9, 4, 5, 6, 15, 14, 6, 5, 23, 9, 6, 4, 5, 20, 5, 9, 6, 20, 10, 19, 6, 14, 15, 3, 6, 5, 23, 9, 6, 16, 15, 3, 29, 19, 27, 12, 12, 38, 28, 53, 31, 6, 58, 34, 51, 31, 11, 12, 49, 23, 9, 6, 43, 15, 19, 21, 6, 15, 14, 6, 23, 2, 4, 6, 18, 15, 22, 10, 5, 3, 21, 37, 4, 6, 19, 9, 20, 5, 23, 6, 20, 10, 19, 6, 5, 23, 9, 6, 17, 3, 2, 10, 18, 9, 24, 12, 49, 23, 20, 5, 6, 16, 2, 29, 29, 6, 4, 5, 20, 10, 19, 6, 20, 10, 19, 6, 16, 2, 10, 41, 4, 6, 5, 23, 9, 6, 14, 3, 2, 9, 10, 19]
 then.

First Citizen:
The shame shall be my death of a horse:
The rest of the state and for the world.

LADY GREY:
The body of his country's death and the prince,
That will stand and wings the friend


In [ ]:
sentence = generate_sentence(net, start_char, temp=3.0)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=10.0)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.8)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.8)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.6)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.5)
print sentence
print charstostr(sentence)